In [4]:
import pandas as pd

# Load the dataset into a DataFrame
file_path = 'globalterrorismdb_0522dist_short.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Filter the DataFrame to remove rows where iyear <= 2001
filtered_df = df[df['iyear'] > 2000]

# Display the first few rows of the filtered DataFrame to verify the changes (optional)
print(filtered_df.head())

filtered_row_count = filtered_df.shape[0]
print(f"Number of rows in the filtered dataset: {filtered_row_count}")

# Save the filtered DataFrame to a new CSV file (optional)
filtered_file_path = 'filtered_dataset.csv'  # Replace with your file path
filtered_df.to_csv(filtered_file_path, index=False)

print(f"Filtered dataset saved to {filtered_file_path}")

C:\Users\tristan\AppData\Local\Temp\ipykernel_38488\593388613.py:5: DtypeWarning: Columns (63,79,94,96,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


          eventid  iyear  imonth  iday approxdate extended resolution  \
712  200101010005   2001       1     1        NaN       No        NaN   
713  200101030006   2001       1     3        NaN       No        NaN   
714  200101040001   2001       1     3        NaN       No        NaN   
715  200101050006   2001       1     5        NaN       No        NaN   
716  200101060005   2001       1     6        NaN       No        NaN   

     country country_txt  region  ...  \
712        6     Algeria      10  ...   
713       45    Colombia       3  ...   
714        6     Algeria      10  ...   
715       45    Colombia       3  ...   
716      182     Somalia      11  ...   

                                              addnotes  \
712                                                NaN   
713                                                NaN   
714                                                NaN   
715  Sources provide different locations for the in...   
716  It is unknown wheth

In [22]:
import os
import re
import fitz
import pytesseract
from PIL import Image
import io
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import TokenTextSplitter

openai_api_key = os.getenv("OPENAI_API_KEY")
neo4j_uri = os.getenv("NEO4J_URI")
neo4j_username = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["NEO4J_URI"] = neo4j_uri
os.environ["NEO4J_USERNAME"] = neo4j_username
os.environ["NEO4J_PASSWORD"] = neo4j_password

graph = Neo4jGraph()
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm_transformer = LLMGraphTransformer(llm=llm)

# List of articles to process
articles = [
    "September 11 attacks",
    "2004 Madrid train bombings",
    "Boston Marathon bombing",
    "2008 Christmas massacres",
    "2008 Mumbai attacks",
    "2015 Ankara bombings",
    "Christchurch mosque shootings",
    "2016 Karrada bombing",
    "2016 Brussels bombings",
    "January 2016 Baghdad–Miqdadiyah attacks"
]

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)

for article in articles:
    print(f"Processing article: {article}")
    
    # Read the Wikipedia article
    raw_documents = WikipediaLoader(query=article).load()
    
    # Split the documents into chunks
    documents = text_splitter.split_documents(raw_documents[:3])
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    graph.add_graph_documents(graph_documents, baseEntityLabel=True, include_source=True)
    
    # Process the documents
    for i, doc in enumerate(documents):
        print(f"Document {i + 1}:")
        print(doc)
        print("---")
    
    print("---")

Processing article: September 11 attacks


d:\Anaconda\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\Anaconda\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


ClientError: {code: Neo.ClientError.Transaction.TransactionHookFailed} {message: You have exceeded the logical size limit of 400000 relationships in your database (attempt to add 40 relationships would reach 400036 relationships). Please consider upgrading to the next tier.}